In [1]:
%pip install requests
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import warnings
import json
import pandas as pd
from dotenv import dotenv_values
from tqdm.auto import tqdm
import time

warnings.filterwarnings('ignore')

USER_AGENT = dotenv_values('.env')['USER_AGENT']

vacancies = pd.DataFrame() #датафрейм с результатом поиска
job_title = "Python разработчик" #название профессии

e:\Educ\ParseHH\hhvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_hh_vacancies(search_text:str, page:int=0):
    '''Функция запроса данных о вакансии с именем serch_name'''
    url = "https://api.hh.ru/vacancies"
    headers = {"User-Agent":USER_AGENT}
    params = {
        "text": search_text,
        "search_field": "name",
        "per_page": 50,
        "page": page
    }
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()  # Возвращаем весь JSON-объект
        
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

In [4]:
def flatten_dict(vacancy, parent_key=[]):
    '''Преобразовывает JSON файл в плоский словарь'''
    new_dict = {}
    for key, value in vacancy.items():
        if isinstance(value, list):
            continue
            
        current_key = parent_key + [key]
        
        if isinstance(value, dict):
            nested = flatten_dict(value, parent_key=current_key)
            new_dict.update(nested)
        else:
            new_dict['_'.join(current_key)] = value
    return new_dict


In [5]:
def add_vacancies(resp_from_hh):
    temp_vac = pd.DataFrame()
    #проходим по всем вакансиям из текущего ответа
    for item in resp_from_hh['items']:
        vacancies_items = flatten_dict(item)
        if temp_vac.empty:
            temp_vac = pd.DataFrame(columns=list(vacancies_items.keys()))
            temp_vac.loc[len(temp_vac)] = vacancies_items

        temp_vac.loc[len(temp_vac)] = vacancies_items
    return temp_vac

In [ ]:
def add_vacancies(resp_from_hh):
    records = []
    for item in resp_from_hh['items']:
        records.append(flatten_dict(item))
    return pd.DataFrame.from_records(records)

In [ ]:
#первый запуск чтобы посмотреть сколько у нас есть страниц и считать первую страницу ответа
hh_response = get_hh_vacancies(job_title)
pages = min(hh_response['pages'], 20)#ограничение api.hh
found = hh_response['found']

In [7]:
pages, found

(14, 675)

In [ ]:
tqdm.pandas()
for num_page in range(pages):
    hh_response = get_hh_vacancies(job_title, page=num_page)
    temp_vac = add_vacancies(hh_response)
    vacancies = pd.concat([vacancies, temp_vac], ignore_index=True)
    time.sleep(0.1)

In [9]:
vacancies

,id,premium,name,department,has_test,response_letter_required,area_id,area_name,area_url,salary_from,...,employer_employer_rating_total_rating,employer_employer_rating_reviews_count,salary_range_frequency_id,salary_range_frequency_name,address_metro,branding_type,branding_tariff,department_id,department_name,employer_logo_urls
0,120236562,False,Junior Python-разработчик,None,False,False,1,Москва,https://api.hh.ru/areas/1,70000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,120236562,False,Junior Python-разработчик,None,False,False,1,Москва,https://api.hh.ru/areas/1,70000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120723284,False,Python-разработчик,None,False,False,4,Новосибирск,https://api.hh.ru/areas/4,40000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,120547195,False,Junior разработчик Python (Django),None,False,False,53,Краснодар,https://api.hh.ru/areas/53,70000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,119970695,False,"Junior python разработчик (Django, Fast API)",None,False,False,1,Москва,https://api.hh.ru/areas/1,40000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,119775921,False,C# разработчик со знанием Python,NaN,False,False,2096,Щербинка (Московская область),https://api.hh.ru/areas/2096,NaN,...,3.8,111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,119765554,False,Программист Python / C / C++ (промышленное и н...,NaN,False,False,1,Москва,https://api.hh.ru/areas/1,NaN,...,4.5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,119746944,False,Ведущий/старший разработчик Python,NaN,False,False,1,Москва,https://api.hh.ru/areas/1,NaN,...,3.8,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,116814467,False,Python-разработчик,NaN,True,False,1641,Набережные Челны,https://api.hh.ru/areas/1641,NaN,...,4.3,367.0,NaN,NaN,NaN,NaN,NaN,68587-68587-manager,Алабуга. Проектный менеджмент,NaN


In [ ]:
#вот так можно оказывается
hh_response_test = get_hh_vacancies(job_title)
test = pd.json_normalize(hh_response_test['items'])

,id,premium,name,department,has_test,response_letter_required,address,response_url,sort_point_distance,published_at,...,address.metro.line_name,address.metro.station_id,address.metro.line_id,address.metro.lat,address.metro.lng,employer.logo_urls,branding.type,branding.tariff,department.id,department.name
0,120236562,False,Junior Python-разработчик,NaN,False,False,NaN,None,None,2025-05-17T15:10:37+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,120723284,False,Python-разработчик,NaN,False,False,NaN,None,None,2025-05-19T16:49:27+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120547195,False,Junior разработчик Python (Django),NaN,False,False,NaN,None,None,2025-05-17T16:30:35+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119970695,False,"Junior python разработчик (Django, Fast API)",NaN,False,False,NaN,None,None,2025-05-19T17:06:06+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,120297920,False,Python разработчик,NaN,False,False,NaN,None,None,2025-05-18T20:33:04+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,120400908,False,Junior Python Backend Developer,NaN,False,False,NaN,None,None,2025-05-18T12:14:31+0300,...,Автозаводская,50.278,50,56.313933,43.994820,NaN,NaN,NaN,NaN,NaN
6,120577562,False,Python developer,NaN,True,False,NaN,None,None,2025-05-18T11:19:53+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,120537190,False,Python-разработчик,NaN,False,False,NaN,None,None,2025-05-17T14:13:51+0300,...,Центральная,49.275,49,55.765346,49.165083,NaN,NaN,NaN,NaN,NaN
8,119787879,False,Junior Python developer,NaN,False,False,NaN,None,None,2025-05-18T18:29:44+0300,...,МЦК,95.532,95,55.690000,37.618889,NaN,NaN,NaN,NaN,NaN
9,120727123,False,Python-разработчик,NaN,False,False,NaN,None,None,2025-05-19T17:53:14+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
